In [2]:
from VC_collections import Collection
from VC_collections.Collection import connect_to_google_drive
from VC_collections.AuthorityFiles import Authority_instance
import gspread

In [64]:
cred = Collection.get_google_drive_credentials()
client = gspread.authorize(cred)
len(client.list_spreadsheet_files())

2314

In [65]:
collection= Collection.Collection("Alma", "Design", "PShHi")

BASE_PATH C:\Users\Yaelg\Google Drive\National_Library\Python\Design\PShHi
C:\Users\Yaelg\Google Drive\National_Library\Python\Design\PShHi\Data 
 C:\Users\Yaelg\Google Drive\National_Library\Python\Design\PShHi\Data\raw 
 C:\Users\Yaelg\Google Drive\National_Library\Python\Design\PShHi\Data\processed 
 C:\Users\Yaelg\Google Drive\National_Library\Python\Design\PShHi\Data\reports 
 C:\Users\Yaelg\Google Drive\National_Library\Python\Design\PShHi\Copyright 
 C:\Users\Yaelg\Google Drive\National_Library\Python\Design\PShHi\Digitization 
 C:\Users\Yaelg\Google Drive\National_Library\Python\Design\PShHi\Authorities 
 C:\Users\Yaelg\Google Drive\National_Library\Python\Design\PShHi\Custom21 
 C:\Users\Yaelg\Google Drive\National_Library\Python\Design\PShHi\Manage18 
 C:\Users\Yaelg\Google Drive\National_Library\Python\Design\PShHi\Custom04\Alma
0 : PShHi_Final_to_Alma_20210531
1 : PShHi_Final_Taub-Levy_23052021
2 : PShHi_Final_to_Alma_20210531
3 : PShHi_Final_to_Alma_20210524
4 : PShHi_With

In [109]:
collection.df_final_data["בעלים נוכחי"].tolist()[0]

'ארכיון אדריכלות ישראל (אא"י)'

In [93]:
values_list = list(set(filter(None,";".join(collection.df_final_data["סוג חומר"].tolist()).split(";"))))

{'תצלומים': 'תצלומים;תצלומים;',
 'תצלום (תצלומים)': 'תצלום;תצלום (תצלומים);צילום;צלום;תמונה;פוטוגרף;תצלומים',
 'תצלום אוויר (תצלומים)': 'תצלום אוויר;תצלום אוויר (תצלומים);תצלום אויר;תצ"א;צלום אויר',
 'תצלום אירוע (תצלומים)': 'תצלום אירוע;תצלום אירוע (תצלומים);צילום אירוע',
 'תצלום חזרה (תצלומים)': 'תצלום חזרה;תצלום חזרה (תצלומים);תצלום שיעור',
 'תצלום חפץ (תצלומים)': 'תצלום חפץ;תצלום חפץ (תצלומים);ציינו את שם וסוג החפץ, ראו דוגמאות בסוגריים: תצלום מודל; חפץ [קופסת טבק, שעון, בובה]; יצירת אמנות [פסל, ציור]; אביזר אפנה [משקפיים, חגורה, ארנק]; פריט לבוש [חולצה, חצאית, מכנסיים]; תכשיט [טבעת, רביד, עגילים]; יודאיקה [מגדל בשמים, כוס קידוש, פמוטים]; אריג [פשתן, צמר, כותנה]; ריהוט [כיסא, שולחן, ארונית]; משחק [לוח, קופסה, חשיבה].',
 'תצלום מסמך (תצלומים)': 'תצלום מסמך;תצלום מסמך (תצלומים);העתק של מסמך;העתק מסמך',
 'תצלום משפחתי (תצלומים)': 'תצלום משפחתי;תצלום משפחתי (תצלומים);',
 'תצלום פורטרט (תצלומים)': 'תצלום פורטרט;תצלום פורטרט (תצלומים);פורטרט;דיוקן',
 'תצלום יח"ץ (תצלומים)': 'תצלום יח"צ;ת

In [104]:
values_list.append("עבודה אקדמית")

In [105]:
from VC_collections.authorities import convert_dict, fix_original

fix_original()

auth_list = convert_dict(Authority_instance.arch_mat_search_dict)
auth_list

TypeError: fix_original() missing 3 required positional arguments: 'col', 'error_words', and 'new_values'

In [106]:
values_not_found = []

for value in values_list:
    if value in auth_list.keys():
        print(f'original {value} is in Authorities')
    else:
        print(f'!!!!!!original {value} not in Authorities') 
        values_not_found.append(value)

original תכתובת (מסמכים מנהליים) is in Authorities
original תצלום (תצלומים) is in Authorities
original חזרה (תמונות נעות) is in Authorities
original קטלוג מורכב (חולפות) is in Authorities
original תכניות הגשה (תכניות ושרטוטים) is in Authorities
original נאום (יצירה ספרותית) is in Authorities
original איור (חומר גרפי/אמנות) is in Authorities
original תכנייה (חולפות) is in Authorities
original סרטי הפקה (תמונות נעות) is in Authorities
original סקיצה (חומר גרפי/אמנות) is in Authorities
original כתבים (יצירה ספרותית) is in Authorities
original סקיצת תלבושות (חומר גרפי/אמנות) is in Authorities
original פרט בניין (תכניות ושרטוטים) is in Authorities
original מפה (פרסומים) is in Authorities
original ציור (חומר גרפי/אמנות) is in Authorities
original כרזה (חולפות) is in Authorities
original פרסום מקוון (פרסומים) is in Authorities
original רשימה (יצירה ספרותית) is in Authorities
original עלון (חולפות) is in Authorities
original מאמר בעיתון יומי - ישראל (פרסומים) is in Authorities
original הודעה ל

In [107]:
auth_list['עבודה אקדמית']

['עבודה אקדמית (מסמכים-כללי)']

In [88]:
len(list(filter(None,";".join(collection.df_final_data["סוג חומר"].tolist()).split(";"))))


1765

In [74]:
with open(r'Data\VIS_907.json') as json_file:
    rosetta_dict = json.load(json_file)

In [ ]:
rosetta_dict

In [ ]:
Collection.collection_field_mapper.keys()

In [ ]:
from VC_collections.AuthorityFiles import Authority_instance
import pprint
from collections import defaultdict

In [ ]:
def convert_dict(new_val2errs):
    d = defaultdict(list)
    for nv, errs in new_val2errs.items():
        if type(errs) == str:
            errs = errs.split(";")
        for err in errs:
            if err != "":
                d[err].append(nv)
    return d

pprint.pprint(convert_dict(Authority_instance.arch_mat_search_dict))

In [ ]:
pprint.pprint(Authority_instance.arch_mat_search_dict)

In [ ]:
Authority_instance.df_media_format_auth[Authority_instance.df_media_format_auth["MEDIA_FORMAT"].str.contains("עץ")]

In [ ]:
import pandas as pd

In [ ]:
privacy_mapping_dict = pd.Series(Authority_instance.df_privacy_values["רמיזות"].values,
                                 index=Authority_instance.df_privacy_values.index.values).to_dict()

In [ ]:
privacy_mapping_dict

In [ ]:
import pandas as pd

In [ ]:
search_dict = pd.Series(Authority_instance.df_privacy_values["רמיזות"].apply(lambda x: x.split(";")), 
                                                               index = Authority_instance.df_privacy_values.index.values).to_dict()

In [ ]:
new_search_dict = {}

for key, value in search_dict.items():
    if len(value)>1:
        for item in value:
            new_search_dict[item] = key
    else:
        new_search_dict[value[0]] = key

In [ ]:
new_search_dict.keys()

In [ ]:
from VC_collections.authorities import convert_dict
convert_dict(new_search_dict)

In [ ]:
for key, value in convert_dict(new_search_dict).items():
    print(f'key:{key}\nvalue:{value}\n')

In [ ]:
Authority_instance.df_arch_mat_search["all_terms"]

In [ ]:
for key, value in Authority_instance.arch_mat_search_dict.items():
    print(f'key:{key}\nvalue:{value}\n')


# split creators

In [ ]:
from VC_collections.AuthorityFiles import Authority_instance
from VC_collections.authorities import find_role, find_name
import pandas as pd

In [ ]:
df = pd.ExcelFile(r'Data/PYaVa_Creators.xlsx').parse("Sheet1")
df = df.set_index("סימול")

In [ ]:
df["יוצר ראשי - איש"] = ''
df["סוג יוצר ראשי - איש"] = ''
df["יוצר ראשי - מוסד"] = ''
df["סוג יוצר ראשי - מוסד"] = ''
df["יוצרים נוספים - איש"] = ''
df["יוצרים נוספים - מוסד"] = ''

In [ ]:

def create_first(creators):

    creator = creators.pop(0)
    
    return find_name(creator).strip(), find_role(creator).strip(), creators

In [ ]:
import numpy as np
roles_not_found = []

for index, row in df.iterrows():

    creators_pers = []
    creators_corps = []
    try:
        creators = row["יוצרים"].split(";")
    except:
        continue
        
    
    
    first_name, first_role, rest_creators = create_first(creators)
    
    if first_role in Authority_instance.roles_dict["pers_roles"]:
        df.loc[index, "יוצר ראשי - איש"] = first_name
        df.loc[index, "סוג יוצר ראשי - איש"] = first_role
    else:
        df.loc[index, "יוצר ראשי - מוסד"] = first_name
        df.loc[index, "סוג יוצר ראשי - מוסד"] = first_role
    
    for creator in rest_creators:
        print(index)
        role = find_role(creator).strip()
        if role in Authority_instance.roles_dict["pers_roles"]:
            print(f'role: {role} is pers')
            creators_pers.append(creator)
        elif role in Authority_instance.roles_dict["corps_roles"]:
            creators_corps.append(creator)
            print(f'role: {role} is corps')
        else:
            roles_not_found.append(creator)
            
    df.loc[index, "יוצרים נוספים - איש"] = ";".join(creators_pers)
    df.loc[index, "יוצרים נוספים - מוסד"] = ";".join(creators_corps)
    if len(roles_not_found) > 0:
        print(roles_not_found)
    

In [ ]:
roles_not_found

In [ ]:
df.to_excel(r'Data/PYaVa_creators_repair.xlsx')

In [ ]:
collection.df_final_data["כותרת ערבית"].empty

In [ ]:
collection.df_final_data.columns

In [ ]:
import pandas as pd

In [ ]:
d= {'ניסיון': ['', '', '']}

In [ ]:
test = pd.DataFrame(d)

In [ ]:

len(test[test["ניסיון"] != ''])

In [9]:
"ArBe" in Authority_instance.df_credits.index

True

In [11]:
Authority_instance.df_credits.loc["ArBe", "מיקום הפקדה עבור בעלים נוכחי"]

'משרד האדריכלים \'בסט אדריכלים ומתכנני ערים בע"מ\''

In [22]:
 test_data_with_dup = {
            "UNITID": ["ABCD", "ABCD", "ASDF"]
        }

In [23]:
df = pd.DataFrame(test_data_with_dup)
df

,UNITID
0,ABCD
1,ABCD
2,ASDF


In [24]:
from VC_collections.columns import dupCheck
import pandas as pd

In [25]:
dupCheck(df, "UNITID")

,UNITID
0,ABCD
1,ABCD


In [21]:

pd.concat(g for _, g in df.groupby("UNITID") if len(g) > 1)

,UNITID
0,ABCD
1,ABCD


In [53]:
import numpy as np
df["test"] = ''

In [54]:
df

,UNITID,test
0,ABCD,
1,ABCD,
2,ASDF,


In [62]:
df["test"].dropna().empty

False

In [61]:
df["test"] = df["test"].dropna()

In [63]:
df

,UNITID,test
0,ABCD,
1,ABCD,
2,ASDF,


In [47]:
df["test"].empty

False

In [110]:
test_008 = "191104k19811981is#xx###############heb#d"

In [114]:
test_008[:7]  + "19841984" + test_008[15:]

'191104k19841984is#xx###############heb#d'

In [118]:
from viapy import api

In [123]:
from urllib.request import urlopen, quote
# if you're running python 3, replace the above with the following:
# from urllib.request import urlopen
# from urllib.parse import quote

def retrieve_viaf_search_results(search_index, search_term, auth_source):
    # url search template formatted to allow easy variable insertion
    search_url_template = 'http://viaf.org/viaf/search/viaf?query=local.{0}+all+{1}+and+local.sources+any+{2}&sortKeys=holdingscount&httpAccept=application/xml'
    
    # since we'll be inserting the three passed variables into the 
    # html search template, we need to make sure to "escape" any
    # special characters. VIAF also requires that they be enclosed in
    # quotes, which themselves have to be escaped. We can use urllib2's 
    # "quote" function for this:
    auth_source = quote(f'"{auth_source}"')
    search_term = quote(f'"{search_term}"')
    
    # build the full search url
    search_url = search_url_template.format(search_index, search_term, auth_source)
    
    # query the url with urlopen and store the returned xml
    response = urlopen(search_url).read()

In [124]:
retrieve_viaf_search_results("persname", "צרפתי, רות")

TypeError: retrieve_viaf_search_results() missing 1 required positional argument: 'auth_source'